#### Code to extract data

**TODO** : 
- lung localizer to crop + window

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

## Imports

In [ ]:
import os
import cv2
import glob
import gdcm
import pydicom
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from tqdm.notebook import tqdm

In [ ]:
from params import *
from data.extraction import *

# Bim

In [ ]:
BIM_PATH = "../input/bimcv_positive_images/"

In [ ]:
paths = glob.glob(BIM_PATH + "*.png")

In [ ]:
meta_bim = pd.DataFrame({"path": paths})

In [ ]:
meta_bim['target'] = 1

In [ ]:
meta_bim.to_csv("../input/meta_bim.csv", index=False)

# Ricord

### Load

In [ ]:
# paths = glob.glob(BIM_PATH + "*.png")

In [ ]:
meta_ricord = pd.read_csv(DATA_PATH + 'MIDRC-RICORD-meta.csv')
meta_ricord = meta_ricord[~meta_ricord['labels'].isna()]

### Labels

In [ ]:
import re

def treat_label(label):
    label = re.sub(',', ' ', label.lower())
    label = label.split(' ')

    labels = np.zeros(4)
    for l in label:
        if l in CLASSES:
            labels[CLASSES.index(l)] += 1
    
    return labels.astype(int)
#     return " ".join(labels.astype(int).astype(str))

In [ ]:
def vote(label):
    n = label.sum()
    
    label = label > n / 2
    
    return label.astype(int)

In [ ]:
meta_ricord['study_labels'] = meta_ricord['labels'].apply(treat_label)
meta_ricord['study_label'] = meta_ricord['study_labels'].apply(vote)

In [ ]:
y = np.stack(meta_ricord['study_label'].values.tolist())
for i, c in enumerate(CLASSES):
    meta_ricord[c] = y[:, i]
    
meta_ricord['unk_label'] = (y.sum(1) == 0)

In [ ]:
RICORD_PATH = "../input/MIDRC-RICORD/MIDRC-RICORD/"

In [ ]:
SAVE_DIR = "../input/MIDRC-RICORD/imgs/"
os.mkdir(SAVE_DIR)

In [ ]:
save_paths = []
for i, path in enumerate(tqdm(meta_ricord['fname'])):
    img = cv2.imread(RICORD_PATH + path)
    
    if len(img.shape) == 3:
        img = img.mean(-1)
    
    img = img - img.min()
    img, window = auto_windowing(img)

    try:
        img, crop_starts = remove_padding(img)
    except:
        pass
        
    img = cv2.resize(img, (SIZE, SIZE), interpolation=cv2.INTER_LINEAR)
    
    if not (i % 100):
        plt.figure(figsize=(9, 9))
        plt.imshow(img, cmap="gray")
        plt.show()
    
    cv2.imwrite(SAVE_DIR + path[:-8] + ".png", img)
    save_paths.append(SAVE_DIR + path[:-8] + ".png")

#     break

In [ ]:
meta_ricord['path'] = save_paths

In [ ]:
meta_ricord.to_csv("../input/meta_ricord.csv", index=False)

# Covidx

In [ ]:
COVIDX_PATH = DATA_PATH + "train/"

In [ ]:
meta_covidx = pd.read_csv(DATA_PATH + "train.txt", sep=" ", names=['path', 'label', 'dataset']).reset_index()

In [ ]:
SAVE_DIR = "../input/covidx_imgs/"
os.mkdir(SAVE_DIR)

In [ ]:
save_paths = []
for i, path in enumerate(tqdm(meta_covidx['path'])):
    img = cv2.imread(COVIDX_PATH + path)

    if len(img.shape) == 3:
        img = img.mean(-1)
    
    img = img - img.min()
    img, window = auto_windowing(img)

#     img, crop_starts = remove_padding(img)
        
    img = cv2.resize(img, (SIZE, SIZE), interpolation=cv2.INTER_LINEAR)
    
    if not (i % 100):
        plt.figure(figsize=(9, 9))
        plt.imshow(img, cmap="gray")
        plt.show()
        
    assert not os.path.exists(SAVE_DIR + path + ".png")
    
    cv2.imwrite(SAVE_DIR + path + ".png", img)
    save_paths.append(SAVE_DIR + path + ".png")


In [ ]:
meta_covidx['path'] = save_paths

In [ ]:
meta_covidx.to_csv("../input/meta_covidx.csv", index=False)